# Partioning the evolution of uncertainties

In [ ]:
import os
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

os.chdir("/home/rooda/Dropbox/Patagonia/MS2 Results/")
cl     = px.colors.qualitative.D3

## Data

In [ ]:
basins_id  = ['PPY', 'PCA', 'NPI-E', 'NPI-W', 'SPI-N', 'SPI-C', 'SPI-S', 'GCN', 'CDI']

var_ens = pd.read_csv("variance_ensemble.csv",      index_col = ["rgi_id", "time"])
var_cli = pd.read_csv("variance_climate.csv",       index_col = ["rgi_id", "time"])
var_geo = pd.read_csv("variance_geometry.csv",      index_col = ["rgi_id", "time"])
var_ssp = pd.read_csv("variance_ssp_scenarios.csv", index_col = ["rgi_id", "time"])
var_gcm = pd.read_csv("variance_gcms.csv",          index_col = ["rgi_id", "time"])
var_nv  = pd.read_csv("variance_ssp_nv.csv",        index_col = "rgi_id")

In [ ]:
var_ssp = var_ssp - var_nv
var_gcm = var_gcm - var_nv

# trick 
var_ssp = var_ssp.where(var_ssp > 0, 0)
var_gcm = var_gcm.where(var_gcm > 0, 0)

var_tot = var_ssp + var_gcm + var_geo + var_cli 

In [ ]:
time_series_ssp = (all_ssps.isel(time = slice(1, 99)).to_dataframe() - all_ssps_nv[variables_final].to_dataframe())
time_series_ssp = time_series_ssp.where(time_series_ssp[variable] > 0, 0)

time_series_gcm = (all_gcms.isel(time = slice(1, 99)).to_dataframe() - all_ssps_nv[variables_final].to_dataframe())
time_series_gcm = time_series_gcm.where(time_series_ssp[variables_final] > 0, 0)

time_series_nv  = all_ssps_nv.to_dataframe()
time_series_mod = all_mods.isel(time = slice(1, 99)).to_dataframe()
time_series_cli = all_climate.isel(time = slice(1, 99)).to_dataframe()

# Figure 9

In [ ]:
variables_final = ["melt_on_glacier", "total_runoff", "smb"]

fig = make_subplots(rows=9, cols=3, horizontal_spacing = 0.05, vertical_spacing = 0.01,
                    shared_xaxes= True, shared_yaxes= False, y_title = "pertantage of total σ (%)", 
                    column_titles = ["Total runoff", "Melt on glacier", "Specific mass balance"], row_titles = basins_id)

for x in range(0,3):
    for y in range(0,9):
        
        var_ssp_id = var_ssp[var_ssp.index.get_level_values('rgi_id') == basins_id[y]][variables_final[x]]
        var_gcm_id = var_gcm[var_gcm.index.get_level_values('rgi_id') == basins_id[y]][variables_final[x]]
        var_mod_id = var_geo[var_geo.index.get_level_values('rgi_id') == basins_id[y]][variables_final[x]]
        var_cli_id = var_cli[var_cli.index.get_level_values('rgi_id') == basins_id[y]][variables_final[x]]
        var_ens_id = var_ens[var_ens.index.get_level_values('rgi_id') == basins_id[y]][variables_final[x]]
        var_tot_id = var_tot[var_tot.index.get_level_values('rgi_id') == basins_id[y]][variables_final[x]]
        
        var_nv_id  = var_gcm_id.copy()
        var_nv_id[basins_id[y]] = float(var_nv[variables_final[x]][var_nv.index == basins_id[y]])
        var_tot_id = var_tot_id + var_nv_id
        
        if x == 0 | y == 0:
            fig.add_trace(go.Scatter(x = var_ssp_id.index.get_level_values(1), y = var_ssp_id, 
                                     mode='lines', name= "SSP", line=dict(color=cl[4], width=1), showlegend=True, legendgroup='g1'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_gcm_id.index.get_level_values(1), y = var_gcm_id, 
                                     mode='lines', name= "GCM", line=dict(color=cl[0], width=1), showlegend=True, legendgroup='g2'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_mod_id.index.get_level_values(1), y = var_mod_id, 
                                     mode='lines', name= "Geometry / Volume", line=dict(color=cl[8], width=1), showlegend=True, legendgroup='g4'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_cli_id.index.get_level_values(1), y = var_cli_id, 
                                     mode='lines', name= "Climate", line=dict(color=cl[2], width=1), showlegend=True, legendgroup='g5'), row=y+1, col=x+1)
            
            fig.add_trace(go.Scatter(x = var_ens_id.index.get_level_values(1), y = var_ens_id, 
                                     mode='lines', name= "Ensemble", line=dict(color="black", width = 1.5), showlegend=True, legendgroup='g6'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_tot_id.index.get_level_values(1), y = var_tot_id, 
                                     mode='lines', name= "Total", line=dict(color="black", width=1, dash = 'dot'), showlegend=True, legendgroup='g7'), row=y+1, col=x+1)

        else:
            fig.add_trace(go.Scatter(x = var_nv_id.index.get_level_values(1), y = var_nv_id, 
                                     mode='lines', name= "Natural variability", line=dict(color=cl[7], width=1), showlegend=False, legendgroup='g3'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_ssp_id.index.get_level_values(1), y = var_ssp_id, 
                                     mode='lines', name= "SSP", line=dict(color=cl[4], width=1), showlegend=False, legendgroup='g1'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_gcm_id.index.get_level_values(1), y = var_gcm_id, 
                                     mode='lines', name= "GCM", line=dict(color=cl[0], width=1), showlegend=False, legendgroup='g2'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_mod_id.index.get_level_values(1), y = var_mod_id, 
                                     mode='lines', name= "Geometry / Volume", line=dict(color=cl[8], width=1), showlegend=False, legendgroup='g4'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_cli_id.index.get_level_values(1), y = var_cli_id, 
                                     mode='lines', name= "Climate", line=dict(color=cl[2], width=1), showlegend=False, legendgroup='g5'), row=y+1, col=x+1)
            
            fig.add_trace(go.Scatter(x = var_ens_id.index.get_level_values(1), y = var_ens_id, 
                                     mode='lines', name= "Ensemble", line=dict(color="black", width = 1.2), showlegend=False, legendgroup='g6'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x = var_tot_id.index.get_level_values(1), y = var_tot_id, 
                                     mode='lines', name= "Total", line=dict(color="black", width=1, dash = 'dot'), showlegend=False, legendgroup='g7'), row=y+1, col=x+1)

fig.update_layout(height = 1200, width = 1000, template = "seaborn")
fig.update_layout(legend = dict(font=dict(size=13), yanchor="top", y=1.03, xanchor="center", x=0.5, orientation = "h"))
fig.for_each_annotation(lambda a:  a.update(y = -0.04) if a.text in ["Total runoff", "Melt on glacier", "Specific mass balance"] else())

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_10_total_contribution.png", height=1200, width=900, scale=4)
fig.show()

# Figure 10

In [ ]:
fig    = make_subplots(rows=9, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.01,
                       shared_xaxes= True, shared_yaxes= True, y_title = "pertantage of total σ (%)", 
                       column_titles = ["Total runoff", "Melt on glacier", "Specific mass balance"], row_titles = basins_id)

for x in range(0,3):
    for y in range(0,9):
        
        var_ssp_id = var_ssp[var_ssp.index.get_level_values('rgi_id') == basins_id[y]]
        var_gcm_id = var_gcm[var_gcm.index.get_level_values('rgi_id') == basins_id[y]]
        var_mod_id = var_geo[var_geo.index.get_level_values('rgi_id') == basins_id[y]]
        var_cli_id = var_cli[var_cli.index.get_level_values('rgi_id') == basins_id[y]]
                
        var_nv_id  = var_gcm_id.copy()
        var_nv_id[basins_id[y]] = float(var_nv[variables_final[x]][var_nv.index == basins_id[y]])

        if x == 0 | y == 0:
            fig.add_trace(go.Scatter(x=var_nv_id.index.get_level_values(1), y=var_nv_id[variables_final[x]], mode='lines', groupnorm='percent', name= "Natural variability", 
                                        line=dict(color=cl[7], width = 1.2), showlegend=True, legendgroup='g5', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_ssp_id.index.get_level_values(1), y=var_ssp_id[variables_final[x]], mode='lines',  groupnorm='percent', name= "SSP", 
                                     line=dict(color=cl[4], width = 1.2), showlegend=True, legendgroup='g1', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_gcm_id.index.get_level_values(1), y=var_gcm_id[variables_final[x]], mode='lines', groupnorm='percent', name= "GCM", 
                                     line=dict(color=cl[0], width = 1.2), showlegend=True, legendgroup='g2', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_mod_id.index.get_level_values(1), y=var_mod_id[variables_final[x]], mode='lines', groupnorm='percent', name= "Area/Volume", 
                                     line=dict(color=cl[8], width = 1.2), showlegend=True, legendgroup='g3', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_cli_id.index.get_level_values(1), y=var_cli_id[variables_final[x]], mode='lines', groupnorm='percent', name= "Climate", 
                                     line=dict(color=cl[2], width = 1.2), showlegend=True, legendgroup='g4', stackgroup='one'), row=y+1, col=x+1)
        
        else:
            fig.add_trace(go.Scatter(x=var_nv_id.index.get_level_values(1), y=var_nv_id[variables_final[x]], mode='lines', groupnorm='percent', name= "Natural variability", 
                                        line=dict(color=cl[7], width = 1.2), showlegend=False, legendgroup='g5', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_ssp_id.index.get_level_values(1), y=var_ssp_id[variables_final[x]], mode='lines',  groupnorm='percent', name= "SSP", 
                                     line=dict(color=cl[4], width = 1.2), showlegend=False, legendgroup='g1', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_gcm_id.index.get_level_values(1), y=var_gcm_id[variables_final[x]], mode='lines', groupnorm='percent', name= "GCM", 
                                     line=dict(color=cl[0], width = 1.2), showlegend=False, legendgroup='g2', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_mod_id.index.get_level_values(1), y=var_mod_id[variables_final[x]], mode='lines', groupnorm='percent', name= "Area/Volume", 
                                     line=dict(color=cl[8], width = 1.2), showlegend=False, legendgroup='g3', stackgroup='one'), row=y+1, col=x+1)
            fig.add_trace(go.Scatter(x=var_cli_id.index.get_level_values(1), y=var_cli_id[variables_final[x]], mode='lines', groupnorm='percent', name= "Climate", 
                                     line=dict(color=cl[2], width = 1.2), showlegend=False, legendgroup='g4', stackgroup='one'), row=y+1, col=x+1)

fig.update_yaxes(range = [0,100])
fig.update_layout(height = 1200, width = 1000, template = "seaborn")
fig.update_layout(legend = dict(font=dict(size=13), yanchor="top", y=1.03, xanchor="center", x=0.5, orientation = "h"))
fig.for_each_annotation(lambda a:  a.update(y = -0.04) if a.text in ["Total runoff", "Melt on glacier", "Specific mass balance"] else())

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_9_rel_contribution.png", height=1200, width=900, scale=4)
fig.show()